In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import os
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModel.from_pretrained("google-bert/bert-base-uncased")
model.to(device)  # Move model to device

# Function to get BERT embeddings for a text
def get_bert_embeddings(text):
    # Tokenize and get input IDs and attention masks
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get embeddings from BERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract last hidden state (embeddings)
    embeddings = outputs.last_hidden_state
    return embeddings

# Function to perform max pooling along the sequence dimension
def max_pooling(embeddings):
    pooled = torch.max(embeddings, dim=1).values
    return pooled

# Define the path to the folder containing CSV files
folder_path = '/content/drive/MyDrive/MiB Dataset/A Fresh Start/User Files Train/subfolder_4/'

# List all CSV files in the folder
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize dictionary to store user-wise pooled outputs
user_pooled_outputs = {}

# Wrap the file processing loop with tqdm to show progress for each file
for file_name in tqdm(files, desc="Processing CSV files"):
    file_path = os.path.join(folder_path, file_name)
    print(f"Processing file: {file_path}")

    try:
        # Load the CSV file
        df = pd.read_csv(file_path)

        # Extract the user ID and text columns
        user_ids = df['user_id'].tolist()  # Replace 'user_id' with the actual column name
        texts = df['text'].tolist()  # Replace 'text' with the actual column name

        # Process each text entry
        for user_id, text in zip(user_ids, texts):
            try:
                # Get embeddings for the text
                embeddings = get_bert_embeddings(text)
                pooled_output = max_pooling(embeddings)
                pooled_output = pooled_output.cpu()  # Move pooled output to CPU

                # Add pooled output to the corresponding user's list
                if user_id not in user_pooled_outputs:
                    user_pooled_outputs[user_id] = []
                user_pooled_outputs[user_id].append(pooled_output)

            except Exception as e:
                print(f"Error processing text: {text[:30]}... Error: {e}")

    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

# Compute the final max-pooled embedding for each user
final_user_embeddings = []

for user_id, outputs in user_pooled_outputs.items():
    if outputs:
        # Stack the user's tweet embeddings into a tensor
        pooled_outputs_tensor = torch.stack(outputs)  # Shape: [num_tweets, embedding_dim]
        print(f"Shape of stacked embeddings for user {user_id}: {pooled_outputs_tensor.shape}")

        # Perform max pooling across tweets (dim=0)
        user_embedding = torch.max(pooled_outputs_tensor, dim=0).values  # Shape: [embedding_dim]
        print(f"Final embedding shape for user {user_id}: {user_embedding.shape}")

        # Add the user's single embedding to the final list
        final_user_embeddings.append(user_embedding)

# Convert final embeddings to a tensor
if final_user_embeddings:
    # Concatenate the embeddings along the first dimension
    final_user_embeddings_tensor_new = torch.cat(final_user_embeddings, dim=0)

    print(f"Shape of final user embeddings: {final_user_embeddings_tensor_new.shape}")

    # Example tensor with shape [no_of_users, 768]
    print(final_user_embeddings_tensor_new.shape)
else:
    print("No user embeddings to aggregate.")

import numpy as n

if final_user_embeddings:
    # Convert the tensor to a NumPy array
    final_user_embeddings_array = final_user_embeddings_tensor_new.cpu().numpy()

    # Save the array to a .npy file
    np.save('/content/drive/MyDrive/MiB Dataset/A Fresh Start/User/final_user_embeddings_tensor_new4.npy', final_user_embeddings_array)
else:
    print("No user embeddings to save.")

Processing CSV files:   0%|          | 0/286 [00:00<?, ?it/s]

Processing file: /content/drive/MyDrive/MiB Dataset/A Fresh Start/User Files Train/subfolder_4/2377007084.0.csv


Processing CSV files:   0%|          | 1/286 [00:03<18:41,  3.94s/it]

Processing file: /content/drive/MyDrive/MiB Dataset/A Fresh Start/User Files Train/subfolder_4/2377530955.0.csv


Processing CSV files:   1%|          | 2/286 [00:06<15:30,  3.28s/it]

...

Processing file: /content/drive/MyDrive/MiB Dataset/A Fresh Start/User Files Train/subfolder_4/2376811856.0.csv


Processing CSV files: 100%|█████████▉| 285/286 [04:47<00:00,  1.39it/s]

Processing file: /content/drive/MyDrive/MiB Dataset/A Fresh Start/User Files Train/subfolder_4/2376714811.0.csv


Processing CSV files: 100%|██████████| 286/286 [04:47<00:00,  1.01s/it]


Shape of stacked embeddings for user 2377007084.0: torch.Size([91, 1, 768])
Final embedding shape for user 2377007084.0: torch.Size([1, 768])
Shape of stacked embeddings for user 2377530955.0: torch.Size([87, 1, 768])
Final embedding shape for user 2377530955.0: torch.Size([1, 768])
Shape of stacked embeddings for user 2380735502.0: torch.Size([92, 1, 768])
Final embedding shape for user 2380735502.0: torch.Size([1, 768])
Shape of stacked embeddings for user 2377373876.0: torch.Size([87, 1, 768])
Final embedding shape for user 2377373876.0: torch.Size([1, 768])
Shape of stacked embeddings for user 2376895730.0: torch.Size([84, 1, 768])
Final embedding shape for user 2376895730.0: torch.Size([1, 768])
Shape of stacked embeddings for user 2379797442.0: torch.Size([82, 1, 768])
Final embedding shape for user 2379797442.0: torch.Size([1, 768])
Shape of stacked embeddings for user 2380062097.0: torch.Size([80, 1, 768])
Final embedding shape for user 2380062097.0: torch.Size([1, 768])
Shape 